![alt text](images/uspas.png)
# VUV and X-ray Free Electron Lasers
# Lab Day 1: Running Genesis with lume-genesis
#### In this session, we will use lume-genesis to do the following:
- load input files
- change input files
- run a genesis simulation
- load output results
- plot output results

The Genesis manual can be found here: http://genesis.web.psi.ch/download.html   
Parts of this notebook taken from lume-genesis examples here: https://github.com/slaclab/lume-genesis/tree/master/examples
##### Authors: N. Neveu, P. Anisimov, D. Nguyen, Y.S. Li
----------

# 1. Import functions we will be using



In [ ]:
import os, h5py
import matplotlib.pyplot as plt
import numpy as np

# from lume-genesis
from genesis import Genesis
from genesis import parsers, lattice
from genesis.parsers import parse_beam_file
from genesis.writers import write_beam_file

----

# 2. Load a lattice file

Genesis lattice files consist of components in the beamline (from manual):

- AW - Main magnetic field (undulator)
- AD - Drift section
- QF - Quadrupole strength
- QX - Quadrupole offset in x
- QY - Quadrupole offset in y
- SL - Solenoid strength
- CX - Corrector strength in x
- CY - Corrector strength in y

In [ ]:
# Read genesis lattice file as elements and parameters
genesis_lat = 'examples/lcls_short.lat'
lat = parsers.parse_genesis_lattice(genesis_lat)
lat.keys()

We have loaded two types of information: 
- eles  = elements
- param = parameters 

Information included in the element descriptions: 
- strength = magnet strength
- L = length of element
- d = distance to previous element

Run the next two commands to access and see the information stored in each key

In [ ]:
# You can index through elements like a normal dict: 
lat['eles'][:3]

In [ ]:
lat['param']

# 3. Run a simple genesis file

In [ ]:
# lume-genesis will create and use a temp directory unless a working dir is specified.
# I prefer to make a new run directory in the current working directory
rundir = os.getcwd()+'/test_run' 
if not os.path.exists(rundir):
    os.makedirs(rundir)

In [ ]:
# Initialize the run, make genesis object with some template input file
gen = Genesis('examples/lcls_short.in', verbose=True, workdir=rundir, use_tempdir=False)

We can now look at what is in the input file and make changes. Note, for this run we need to reduce the number of slices so that the run can finish on a laptop. All values in G are parameters in the input file. Take a look at the information from the input file in the following cells:

In [ ]:
# All input are fields from input file
# if no value in input file template, default is filled in
gen.input.keys()

In [ ]:
# list of elements and parameters
# s = end of elements, final positions
# lume-genesis fills in empty spaces for overlap
gen.input['lattice'].keys()

In [ ]:
# These can also be called w/o key notation:
gen.beam     # no beam is loaded yet (empty)
gen.lattice  # gives same output as call in last cell
gen.param.keys()

# Pick a few parameters each, and find them in the manual.
# Summarize what they are used for. 

Genesis can output many types of data. There are several flags for data output, as shown in next code blocks:

- idump - wavefront output
- idmpfld - 
- idmppar - 
- ippart - 
- ipradi - 
- SL - Solenoid strength
- CX - Corrector strength in x
- CY - Corrector strength in y

In [ ]:
# Dump wavefront output
# set to 0 for no wavefront output
gen.param['idump'] = 1

In [ ]:
# Simulation inputs can be changed before running the simulation

# Change number of slices and set it to 256 in the next cell
# G['nslice'] = 256

# Here are some commonly used parameters: 

# # Turn on field output
# G['idmpfld'] = 1

# # Turn on particle output
# G['idmppar'] = 1

# Change number of particles
# G['npart'] = 2048

# # Turn on history
# G['ippart'] = 10
# G['ipradi'] = 0

# Again, all of these keys come from the input file

In [ ]:
# You can save the updated input file
gen.write_input()

# 4. Running sims and archiving the data

To save on time, we'll load the data that was previously saved. Commands needed to run the simulation and archive the files are shown as comments here. Saving the data to an h5 file allows you to reload with lume-genesis easily, and only have to save one file. You can return to your data after closing jupyterlab, and don't need to leave the browser window open.

Some notes on output files:  
fld - not parsed here, field history file (can get huge)  
par - can get very large if many slices  
dlf (wavefront), dpa (phase space) are final field and particle files





In [ ]:
# This command will run the genesis simulation 
# We're skipping this part for now
# gen.binary_prefixes = ['mpirun', '-n', '10']
# gen.run()

In [ ]:
# G.output['run_info'] # gives info about run time and location
# Example output: 
# {'start_time': 1608185282.456321,
#  'run_script': '/Users/nneveu/Code/anaconda3/envs/lume/bin/genesis2 genesis.in',
#  'run_time': 1231.4947772026062,
#  'run_error': False}

In [ ]:
# Archive data to h5 so that you can reload it later
# gen.archive('lcls_test.h5')

In [ ]:
# This will only work if 'idump' option is non-zero
# G.write_wavefront() # save wavefront data to seperate h5 file

# 5. Loading data from archive file & looking at output

In [ ]:
gen.load_archive('examples/lcls_short.h5')
# Output data is now saved in G object

In [ ]:
gen.output.keys() # three categories of saved data

In [ ]:
gen.output['run_info'] #information about simulation run time and location

In [ ]:
gen.output['param'].keys() # input parameters to simulation
# should be similar or same as input params
# some numbers filled in during run, slight changes from input

In [ ]:
# These are the available data types
gen.output['data'].keys() # all the output data

----

# 6. Plotting results

In [ ]:
# Get z values in sim
zlist = G.output['data']['z'] # 1D array
zlist.shape

In [ ]:
# Get power. This is a 2d array of: slice, z
power = G.output['data']['power']
power.shape

In [ ]:
plt.plot(zlist,np.log(power[0]))
plt.xlabel('z along bunch?')
plt.ylabel('Power (W)')
plt.show()

In [ ]:
wf  = h5py.File('examples/lcls_short_wavefront.h5', 'r')
dfl = wf['data/000000/meshes/electricField/x']
wf['data/000000/meshes/electricField/x']

In [ ]:
# Get parameters from .out file
params = G.output['param']
my_ncar = params['ncar']
my_dgrid = params['dgrid']

my_nz = 1

# Field phase at end, slice 0
def plot_field(dat, dgrid):
    ndat = np.angle(dat)
    plt.imshow(ndat, extent = [1000*dgrid*i for i in [-1,1,-1,1]])
    plt.xlabel('x (mm)')
    plt.ylabel('y (mm)')
    plt.show()
    
plot_field(dfl[:, :, 0], my_dgrid )

In [ ]:
plot_field(dfl[:, :, 100], my_dgrid )

In [ ]:
plot_field(dfl[:, :, 200], my_dgrid )

In [ ]:
plot_field(dfl[:, :, 300], my_dgrid )

In [ ]:
plot_field(dfl[:, :, 650], my_dgrid )

----

----